In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycytominer
import easygui as eg
import sys
import hdbscan

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.decomposition import PCA

sys.path.append(r"C:\Users\Fer\Documents\GitHub")
from scripts_notebooks_fossa.pycombat_umap import combat_util

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# 0. Inputs

In [ ]:
myfile = eg.fileopenbox(msg="Choose a file", default=r"F:")
print('Filename', myfile)
df = pd.read_csv(myfile)
df.head()

In [ ]:
n_neighbors_input = 15
min_dist_input = 0.5
metric = 'cosine'
hover_list = ['Metadata_Plate','Metadata_Well', 'Metadata_compound', 'Metadata_concentration_uM']
number_of_iterations=50
true_labels = "Metadata_compound"
size_col="Metadata_NPSize_nm"

In [ ]:
cols_to_join = ["Metadata_compound", "Metadata_concentration_uM"]
df, new_col = combat_util.col_generator(df, cols_to_join = cols_to_join)

# #just remove the 0 for the non-treated wells
# df[new_col] = df[new_col].str.replace(r' 0', ' 20', regex=True)
# df[new_col].unique()

## Filter out treatments

If you've performed any profile evaluation and knows which profiles are not technically reproducible.

In [ ]:
filter_out=['Orphenadrine 1', 'Non-treated 0', 'Lactose 1', 'Lactose 10']

In [ ]:
df_filtered = df.query(f'{new_col} not in {filter_out}').reset_index(drop=True)

# 1. Prepare X and Y and UMAP vectors

In [ ]:
meta = pycytominer.cyto_utils.features.infer_cp_features(df_filtered, metadata=True)
feat = [x for x in df_filtered.columns.tolist() if x not in meta]
X = pd.DataFrame(df_filtered, columns=feat)
y = pd.DataFrame(df_filtered, columns=meta)

In [ ]:
df_3d = combat_util.generate_x_y_umap(df_filtered, n_neighbors=n_neighbors_input, 
                                                  min_dist=min_dist_input, metric=metric, iterate=True, 
                                                  number_runs=number_of_iterations, n_components=3)

In [ ]:
classical_plot = combat_util.plot_umap_3d(df_3d, color_col='Metadata_compound', 
                    #   split_df = False, split_column = None, np = None,
                      hover_cols=hover_list,
                      size=True, size_col=size_col,
                      # x="0", y="1",
                      # error_x="x_err", error_y="y_err",
                       dili_color=True
                      )

# 2. PCA clustering, then HDSBCAN

For the sake of performance we’ll reduce the dimensionality of the data down to 50 dimensions via PCA (this recovers most of the variance), since HDBSCAN scales somewhat poorly with the dimensionality of the data it will work on.

In [ ]:
pca = PCA(n_components=30)
lowd_df = pca.fit_transform(X)
hdbscan_labels = hdbscan.HDBSCAN(min_cluster_size=7).fit_predict(lowd_df)

- The resulting array contains the cumulative percentage of variance explained by the first i principal components, where i ranges from 1 to k.

In [ ]:
# Get the percentage of variance explained by each component
variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative sum of the explained variance ratios
cumulative_variance_ratio = np.cumsum(variance_ratio)

# Plot the cumulative explained variance ratio
plt.plot(cumulative_variance_ratio)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.show()

-  it should be noted that one of the features of HDBSCAN is that it can refuse to cluster some points and classify them as “noise”.

In [ ]:
df_3d["Metadata_hdbscan_label"] = hdbscan_labels

classical_plot
combat_util.plot_umap_3d(df_3d, color_col='Metadata_hdbscan_label', 
                    #   split_df = False, split_column = None, np = None,
                      hover_cols=hover_list,
                      size=True, size_col = size_col,
                      # x="0", y="1",
                      # error_x="x_err", error_y="y_err",
                       dili_color=True
                      )

## 2.2 Evaluate

### 0 representing a bad (essentially random) clustering and 1 representing perfectly recovering the true labels.

In [ ]:
(
    adjusted_rand_score(y[true_labels], hdbscan_labels),
    adjusted_mutual_info_score(y[true_labels], hdbscan_labels)
)

### We can instead only look at the subset of the data that HDBSCAN was actually confident enough to assign to clusters – a simple sub-selection will let us recompute the scores for only that data.

In [ ]:
clustered = (hdbscan_labels >= 0)
(
    adjusted_rand_score(y[true_labels][clustered], hdbscan_labels[clustered]),
    adjusted_mutual_info_score(y[true_labels][clustered], hdbscan_labels[clustered])
)

### How much of the data did HDBSCAN actually assign to clusters? 

In [ ]:
np.sum(clustered) / X.shape[0]

## 2.3 Inside HDBSCAN hierarchy

In [ ]:
clusterer = hdbscan.HDBSCAN(min_samples=6, min_cluster_size=10, metric='euclidean').fit(result_array)

We can now see the hierarchy as a dendrogram, the width (and color) of each branch representing the number of points in the cluster at that level. If we wish to know which branches were selected by the HDBSCAN* algorithm we can pass select_clusters=True. You can even pass a selection palette to color the selections according to the cluster labeling.

In [ ]:
clusterer.condensed_tree_.plot(select_clusters=True)

In [ ]:
clusterer.single_linkage_tree_.plot()

# 3. UMAP clustering, then HDBSCAN

### Select the UMAP vectors

In [ ]:
# Select the three columns
selected_columns = df_3d[['0', '1', '2']]

# Convert to a NumPy array
result_array = selected_columns.to_numpy()

## 3.1 Run with cosine metric

Uncomment the lines below

In [ ]:
# from scipy.spatial import distance

# # define X (n_samples, n_features)
# mat = distance.cdist(result_array, result_array, metric='cosine')
# hdb = hdbscan.HDBSCAN(min_samples=6,min_cluster_size=10,metric='precomputed')
# hdbscan_labels=hdb.fit_predict(mat)

## 3.1 OR run with euclidean metric

In [ ]:
hdbscan_labels = hdbscan.HDBSCAN(min_samples=6, min_cluster_size=10, metric='euclidean').fit_predict(result_array)

In [ ]:
df_3d["Metadata_hdbscan_label"] = hdbscan_labels


classical_plot
combat_util.plot_umap_3d(df_3d, color_col='Metadata_hdbscan_label', 
                    #   split_df = False, split_column = None, np = None,
                      hover_cols=hover_list,
                      size=True, size_col = size_col,
                      # x="0", y="1",
                      # error_x="x_err", error_y="y_err",
                       discrete=True
                      )


## 3.2 Evaluate

### 0 representing a bad (essentially random) clustering and 1 representing perfectly recovering the true labels.

In [ ]:
(
    adjusted_rand_score(y[true_labels], hdbscan_labels),
    adjusted_mutual_info_score(y[true_labels], hdbscan_labels)
)

### We can instead only look at the subset of the data that HDBSCAN was actually confident enough to assign to clusters – a simple sub-selection will let us recompute the scores for only that data.

In [ ]:
clustered = (hdbscan_labels >= 0)
(
    adjusted_rand_score(y[true_labels][clustered], hdbscan_labels[clustered]),
    adjusted_mutual_info_score(y[true_labels][clustered], hdbscan_labels[clustered])
)

### How much of the data did HDBSCAN actually assign to clusters? 

In [ ]:
np.sum(clustered) / X.shape[0]

## 3.3 Inside HDBSCAN hierarchy

In [ ]:
clusterer = hdbscan.HDBSCAN(min_samples=6, min_cluster_size=10, metric='euclidean').fit(result_array)

We can now see the hierarchy as a dendrogram, the width (and color) of each branch representing the number of points in the cluster at that level. If we wish to know which branches were selected by the HDBSCAN* algorithm we can pass select_clusters=True. You can even pass a selection palette to color the selections according to the cluster labeling.

In [ ]:
clusterer.condensed_tree_.plot(select_clusters=True)

In [ ]:
clusterer.single_linkage_tree_.plot()

# 4. kmeans clustering for comparison

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42, n_init=20, algorithm="elkan")
kmeans_labels = kmeans.fit_predict(result_array)

In [ ]:
df_3d["Metadata_kmeans_label"] = kmeans_labels

classical_plot
combat_util.plot_umap_3d(df_3d, color_col='Metadata_kmeans_label', 
                    #   split_df = False, split_column = None, np = None,
                      hover_cols=hover_list,
                      size=True, size_col=size_col,
                      # x="0", y="1",
                      # error_x="x_err", error_y="y_err",
                       discrete=True
                      )

### Interpretation:

ARI = 1: Perfect clustering.

ARI = 0: Clustering is no better than random.

ARI = -1: Perfect disagreement between true and predicted labels.

In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Assuming y_true is the true cluster assignments and kmeans_labels is the predicted labels from K-means
ari = adjusted_rand_score(y[true_labels], kmeans_labels)
nmi = normalized_mutual_info_score(y[true_labels], kmeans_labels)

print("Adjusted Rand Index:", ari)
print("Normalized Mutual Information:", nmi)